In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory

2023-05-29 16:09:21.080736: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 16:09:21.893199: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
saved_model = '/datadrive/AACS2023/ESPCN/ESPCN_model/'

In [10]:
def get_model(upscale_factor=3, channels=1):
    conv_args = {
        "activation": "relu",
        "kernel_initializer": "Orthogonal",
        "padding": "same",
    }
    # run["conv_args"] = conv_args
    inputs = keras.Input(shape=(None, None, channels))
    x = layers.Conv2D(64, 5, **conv_args)(inputs)
    x = layers.Conv2D(64, 3, **conv_args)(x)
    x = layers.Conv2D(32, 3, **conv_args)(x)
    x = layers.Conv2D(channels * (upscale_factor ** 2), 3, **conv_args)(x)
    outputs = tf.nn.depth_to_space(x, upscale_factor)

    return keras.Model(inputs, outputs)

In [12]:
model = get_model(upscale_factor=4, channels=1)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, None, 1)]   0         
                                                                 
 conv2d_8 (Conv2D)           (None, None, None, 64)    1664      
                                                                 
 conv2d_9 (Conv2D)           (None, None, None, 64)    36928     
                                                                 
 conv2d_10 (Conv2D)          (None, None, None, 32)    18464     
                                                                 
 conv2d_11 (Conv2D)          (None, None, None, 16)    4624      
                                                                 
 tf.nn.depth_to_space_2 (TFO  (None, None, None, 1)    0         
 pLambda)                                                        
                                                           

In [13]:
model.load_weights(saved_model)

2023-05-29 16:13:57.654436: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open /datadrive/AACS2023/ESPCN/ESPCN_model/: FAILED_PRECONDITION: /datadrive/AACS2023/ESPCN/ESPCN_model; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [19]:
import coremltools as ct

mlmodel = ct.convert(
                    model,
                    source='tensorflow',
                    convert_to='mlprogram',
                    inputs=[ct.TensorType(shape=(1, 64, 64, 1))])


2023-05-29 16:22:02.515169: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 4
2023-05-29 16:22:02.515701: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-29 16:22:02.523131: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14587 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0001:00:00.0, compute capability: 7.0
2023-05-29 16:22:02.523452: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14587 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0002:00:00.0, compute capability: 7.0
2023-05-29 16:22:02.523762: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 14587 MB memory:  -> device: 2, name: Tesla V100-PCIE-16GB, pci bus id: 0003:00:

In [ ]:
mlmodel.save('ESPCN.mlpackage')